In [ ]:
from ast import Str
from genericpath import exists
import string
from tracemalloc import stop
from turtle import left
from matplotlib.pyplot import title
from matplotlib.style import use
import jieba
import json
from collections import OrderedDict
from datetime import datetime
from zhon import hanzi
import os
import time
import pandas as pd
import re
from price_preprocess import get_company_name_code, new_left_company

In [ ]:
import tqdm

In [ ]:
company_path = 'D:\毕设code\\news_data_from2016to2021\companies\深交所A股列表_主板.xlsx'
def get_company_name_code(company_path):
    company_detail = pd.read_excel(company_path)
    name_code = company_detail.loc[:, ['公司全称', 'A股代码', 'A股简称']]
    #代码部分是数字所以还需要做一些处理,使用内置的zfill函数
    name_code['A股代码'] = pd.DataFrame(
        [str(code).zfill(6) for code in name_code['A股代码'].values],
        columns=['A股代码'])
    #股票简称支付中间存在空格
    name_code['A股简称'] = pd.DataFrame(
        [name.replace(' ', '') for name in name_code['A股简称'].values],
        columns=['A股简称'])

    return zip(name_code['公司全称'].values, name_code['A股代码'].values,
               name_code['A股简称'])

In [ ]:
#新闻所在路径
news_path = 'D:\毕设code\\news_data_from2016to2021'
#年份列表
year_list = ['2016', '2017', '2018', '2019', '2020', '2021']
#文件名称
file_name = 'ER_NewsInfo.xlsx'
#需要额外保存的列名
columns_name = ['DeclareDate', 'Classify', 'Title', 'NewsContent', 'Symbol']
#交易日数据
trade_date_path = 'D:\毕设code\\news_data_from2016to2021\companies\\trade_cal_clean.csv'
#中文停词
chnstopword = 'D:\chrome\Listed-company-news-crawl-and-text-analysis-master\src\Leorio\chnstopwords.txt'
#储存最后使用的公司的路径
company_final = 'D:\毕设code\\news_data_from2016to2021\companies\\final_use'

In [ ]:
def trade_day(trade_day_path):
    print('获取了交易日数据')
    return pd.read_csv(trade_day_path)
def date_trans(date):
    date = pd.to_datetime(date)
    return date.strftime('%Y-%m-%d')
def company_list(tripule):
    company_name, company_code, company_short = [], [], []
    for elem in tripule:
        company_name.append(elem[0])
        company_code.append(elem[1])
        company_short.append(elem[2])
    print('获得了公司信息')
    return company_name, company_code, company_short
def get_news_data(year):
    start_time = time.time()
    news_name = os.path.join(news_path, str(year), file_name)
    news_data = pd.read_excel(news_name)  #获取新闻资讯数据
    end_time = time.time()
    print(end_time-start_time)
    news_data['DeclareDate'] = news_data['DeclareDate'].apply(
        lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d'
                                                                     ))
    return news_data[columns_name]

def get_specific_date_news(date, news_data):
    date = date_trans(date)
    return news_data[news_data['DeclareDate'] == date]


In [ ]:
# 让文本只保留汉字
def is_chinese(uchar):
    if uchar >= u'\u4e00' and uchar <= u'\u9fa5':
        return True
    else:
        return False
def format_str(content):
    content_str = ''
    for i in content:
        if is_chinese(i):
            content_str = content_str + i
    return content_str

In [ ]:
def filter_str(desstr, restr=''):
    res = re.compile("[^\u4e00-\u9fa5^a-z^A-Z^0-9]")
    return res.sub(restr, desstr)

In [ ]:
def remove_braceket(text):
    #title = re.sub('^《[\u4e00-\u9fa5_a-zA-Z0-9]+》','',title) 法律条令
    text = re.sub('^([\u4e00-\u9fa5_a-zA-Z0-9]+)', '', text)
    text = re.sub('^【[\u4e00-\u9fa5_a-zA-Z0-9]+】', '', text)
    text = re.sub('^@[\u4e00-\u9fa5_a-zA-Z0-9]+', '', text)
    return text
def title_preprocess(title):
    title_list = title.split()
    title_list = [
        remove_braceket(title_constitution)
        for title_constitution in title_list
    ]
    title_list = [
        format_str(title_constitution) for title_constitution in title_list
    ]
    return ''.join(title_list)
#todo，目前发现主要是链接，之后可能会有其他操作
def content_preprocess(content):
    return re.sub('^([a-zA-z]+://[^\s]*)', '', content)
def split_sentence(paragraph):
    print('分句ing')
    sentence_set = re.findall(hanzi.sentence, paragraph)
    return sentence_set
def news_preprocess(content):
    """去除中文的标点符号

    Args:
        content (str): 中文字符串

    Returns:
        [list]: 去除标点符号之后的句子列表
    """
    content = content.rsplit('（', 1)[0]  #去掉'（文章来源：xxxx）'
    content = content.replace('\u3000', ' ')  #首先将所有的全角空格替换掉
    content_split_list = content.split()  #此处只是得到段落，还需要处理
    content_list = []
    for paragraph in content_split_list:
        sentence_set = split_sentence(paragraph)  #段落的句子集合，包含奇奇怪怪的比如emoji等符号
        sentence_set = list(map(filter_str, sentence_set))  #去除奇怪符号
        content_list += sentence_set  #这个地方事实上没有考虑段落这一关系，这个地方之后如有需要可以改
        #content_list.append(sentence_set) 即可，就是段落区分，这个之后的处理也需要一点变化，先待定。
    chinese_list = []  #一条资讯全部句子
    for sentence in content_list:
        chinese_list.append(format_str(sentence))
    return chinese_list


In [ ]:
def single_company_news(company_name, company_code, company_short, date, news_data):
    """获取新闻标题，内容以及资讯分类等信息

    Args:
        company_name (str): 全称
        company_code (str): 代码
        company_short (str): 简称
        date (date): 需要的某天资讯 
        year (str): 某一年的资讯

    Returns:
        [tuple]: 需要的信息的元组
    """
    #获取指定日期全部新闻
    specific_date_news = get_specific_date_news(date, news_data)
    #去除newscontent为空的数据行
    specific_date_news = specific_date_news.dropna(subset=['NewsContent','Title'])
    #对相关联股票代码这一列进行填充操作
    specific_date_news = specific_date_news.fillna({'Symbol': '000000'})  
    #index重置，这个很有必要
    specific_date_news = specific_date_news.reset_index(drop=True)
    # title列和newscontent列可能出现数字类型，这个需要进行处理
    specific_date_news[['Title','NewsConten']] = specific_date_news[['Title','NewsContent']].astype(str)
    news_title, news_content, news_classification = [], [], []
    exists_news = False
    #获取某公司的相关新闻
    for idx in range(len(specific_date_news)):
        if specific_date_news.loc[idx,'Title'].find(company_name) != -1 or \
            specific_date_news.loc[idx,'NewsContent'].find(company_name) != -1 or \
            specific_date_news.loc[idx,'Title'].find(company_code) != -1 or \
            specific_date_news.loc[idx,'NewsContent'].find(company_code) != -1 or \
            specific_date_news.loc[idx,'Title'].find(company_short) != -1 or \
            specific_date_news.loc[idx,'NewsContent'].find(company_short) != -1 or \
            company_code == specific_date_news.loc[idx,'Symbol']:
            # 注意一下symbol列的情况。
            title = title_preprocess(specific_date_news.loc[idx, 'Title'])
            content = content_preprocess(specific_date_news.loc[idx,
                                                                'NewsContent'])
            news_title.append(title)
            news_content.append(content)
            news_classification.append(specific_date_news.loc[idx, 'Classify'])
    if len(news_title) != 0:
        exists_news = True
    return exists_news, news_title, news_content, news_classification


In [ ]:
def company_detail(company_name, company_code, company_short):
    company = OrderedDict()
    company['name'] = company_name
    company['code'] = company_code
    company['short'] = company_short
    return company
def maintain_relevent_news_single_year(year, company_final, trade_date_path):
    tripule = get_company_name_code(company_path)
    company_name, company_code, company_short = [], [], []
    company_left = new_left_company(company_final)  #获取留下来的公式code
    company_left_code = list(map(lambda x: x.replace('.csv', ''),
                                 company_left))
    #获取到留下的公司的代码，简称等信息
    for elem in tripule:
        if elem[1] in company_left_code:
            company_name.append(elem[0])
            company_code.append(elem[1])
            company_short.append(elem[2])
    #获取交易日数据
    trade_day_ = trade_day(trade_date_path)
    trade_day_['year'] = trade_day_['cal_date'].apply(
        lambda x: str(x)[:4])  #获取年份
    trade_day_['date'] = trade_day_['cal_date'].apply(
        lambda x: date_trans(str(x)))
    trade_day_data = trade_day_[trade_day_['year'] == str(year)]
    trade_day_data = trade_day_data.reset_index(drop=True)  #获取指定年份的交易日数据
    total_dict = OrderedDict()
    news_data = get_news_data(year)
    print('----开始----')
    for index in range(len(trade_day_data)-2,len(trade_day_data)):
        news_dict = OrderedDict()
        is_open = trade_day_data.loc[index, 'is_open']  #获取该日期是否开市
        news_dict['is_open'] = is_open
        company_data = OrderedDict()
        for i in range(len(company_code)):
            company_dict = company_detail(company_name[i], company_code[i],
                                          company_short[i])
            exists_news, news_title, news_content, news_classification = single_company_news(
                company_name[i], company_code[i], company_short[i],
                trade_day_data.loc[index, 'date'], news_data)
            company_dict['exists_news'] = exists_news
            company_dict['title'] = news_title
            company_dict['content'] = news_content
            company_dict['classification'] = news_classification
            company_data[company_code[i]] = company_dict
        news_dict['company'] = company_data
        total_dict[trade_day_data.loc[index, 'date']] = news_dict
        print('完成{}的处理！'.format(trade_day_data.loc[index,'date']))

    json_str = json.dumps(total_dict, indent=4)
    with open(os.path.join('news_data_from2016to2021\companies',
                           str(year) + '_data.json'), 'w') as json_file:
        json_file.write(json_str)
    print('----完成----')


def maintain_five_year(year_list, company_final, trade_date_path):
    """将五年的数据进行分类保存

    Args:
        year_list (list): 年份的列表，[2020,2021]这种
        company_final (str): 路径
        trade_date_path (str): 交易数据路径
    """
    for year in year_list:
        maintain_relevent_news_single_year(year, company_final,
                                           trade_date_path)
        print('完成一年的数据分类')

In [ ]:
year_list = [2018]
maintain_five_year(year_list, company_final, trade_date_path)

In [23]:
news_data = get_news_data(2019)

119.39613699913025


In [24]:
news_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 445396 entries, 0 to 445395
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   DeclareDate  445396 non-null  object
 1   Classify     445396 non-null  object
 2   Title        445396 non-null  object
 3   NewsContent  445391 non-null  object
 4   Symbol       150442 non-null  object
dtypes: object(5)
memory usage: 17.0+ MB


In [25]:
news_data['DeclareDate']

0         2019-01-01
1         2019-01-01
2         2019-01-01
3         2019-01-01
4         2019-01-01
             ...    
445391    2019-12-30
445392    2019-12-30
445393    2019-12-30
445394    2019-12-30
445395    2019-12-30
Name: DeclareDate, Length: 445396, dtype: object

In [28]:
news_data = news_data.drop(index = news_data.loc[news_data['DeclareDate'] == '2019-01-02'].index)
news_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 444041 entries, 1355 to 445395
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   DeclareDate  444041 non-null  object
 1   Classify     444041 non-null  object
 2   Title        444041 non-null  object
 3   NewsContent  444036 non-null  object
 4   Symbol       149871 non-null  object
dtypes: object(5)
memory usage: 16.9+ MB
